In [1]:
import torch
torch.cuda.empty_cache() # Ekran kartı belleğini temizle
print(torch.cuda.is_available())

False


/home/muti/Documents/GitHub/Syscall-Anomaly-Detection/.venv/lib/python3.13/site-packages/torch/cuda/__init__.py:184: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import os
import torch
from torch.utils.data import Dataset

class SyscallDataset(Dataset):
    def __init__(self, normal_dir, attack_dir):
        """
        normal_dir: Training_Data_Master klasörünün yolu
        attack_dir: Attack_Data_Master klasörünün yolu
        """
        self.data_paths = []
        self.labels = []
        
        # Normal verileri yükle (Etiket: 0)
        self._load_files(normal_dir, label=0)
        
        # Saldırı verilerini yükle (Etiket: 1)
        self._load_files(attack_dir, label=1)
        
    def _load_files(self, directory, label):
        # Klasör içindeki tüm .txt dosyalarını bulup listeye ekleriz
        for root, _, files in os.walk(directory):

            for file in sorted(files): # Dosyaları alfabetik sıralayalım ki train/test ayırırken karışmasın
                
                if file.endswith('.txt'):
                    self.data_paths.append(os.path.join(root, file))
                    self.labels.append(label)

    def __len__(self):
        # Toplam dosya sayısını döndürür
        return len(self.data_paths)

    def __getitem__(self, idx):
        # İstenen dosyanın yolunu al
        file_path = self.data_paths[idx]
        label = self.labels[idx]
        
        # Dosyayı oku ve içindeki string sayıları integer listesine çevir
        with open(file_path, 'r') as f:
            content = f.read().strip().split()
            # String dizisini integer dizisine dönüştür (List Comprehension)
            # content içindeki her x için: integer yap ve +1 ekle
            # Böylece orijinal dosyadaki '0' -> '1', '1' -> '2' olur.
            syscall_sequence = [int(x) + 1 for x in content if x.isdigit()][:500] # İlk 500'ü al
            #syscall_sequence = [int(x) for x in content if x.isdigit()]
            
        # PyTorch modelinin anlayabilmesi için listeyi Tensor'a çevir
        # Syscall ID'leri tam sayı olduğu için dtype=torch.long kullanıyoruz
        sequence_tensor = torch.tensor(syscall_sequence, dtype=torch.long)
        label_tensor = torch.tensor(label, dtype=torch.float32)
        
        return sequence_tensor, label_tensor

In [3]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, random_split , Subset
import random


def collate_fn_pad(batch):
    sequences = []
    labels = []
    
    for seq, label in batch:
        sequences.append(seq)
        labels.append(label)
        
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)
    labels_tensor = torch.stack(labels).unsqueeze(1)
    
    return padded_sequences, labels_tensor



# Tüm veriyi yükle
full_dataset = SyscallDataset('../data/Training_Data_Master', '../data/Attack_Data_Master')
# Sınıflara göre indeksleri ayır
normal_indices = [i for i, l in enumerate(full_dataset.labels) if l == 0]
attack_indices = [i for i, l in enumerate(full_dataset.labels) if l == 1]
# Her sınıfın kendi içinde karıştır
random.shuffle(normal_indices)
random.shuffle(attack_indices)
# Her sınıftan %80 eğitim, %20 test al
def split_indices(indices, ratio=0.8):
    cut = int(ratio * len(indices))
    return indices[:cut], indices[cut:]
normal_train, normal_test = split_indices(normal_indices)
attack_train, attack_test = split_indices(attack_indices)
# Birleştir
train_indices = normal_train + attack_train
test_indices  = normal_test  + attack_test
# Subset olarak sar
train_dataset = Subset(full_dataset, train_indices)
test_dataset  = Subset(full_dataset, test_indices)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,  collate_fn=collate_fn_pad)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, collate_fn=collate_fn_pad)
# İstatistikleri yazdır
normal_count = len(normal_indices)
attack_count = len(attack_indices)
print(f"Toplam  → Normal: {normal_count} | Saldırı: {attack_count}")
print(f"Eğitim  → Normal: {len(normal_train)} | Saldırı: {len(attack_train)}")
print(f"Test    → Normal: {len(normal_test)}  | Saldırı: {len(attack_test)}")

Toplam  → Normal: 833 | Saldırı: 746
Eğitim  → Normal: 666 | Saldırı: 596
Test    → Normal: 167  | Saldırı: 150


# İşlem,Nerede Yapılır?,Teknik Rolü
Sıfırları Ekleme,collate_fn_pad,Dizileri aynı boyuta getirip matris oluşturmak.


Sıfırları Yok Sayma,SyscallLSTM (padding_idx),Eklenen sıfırların öğrenme sürecini bozmamasını sağlamak.

# 1. Dosya Yollarını Belirle (src klasöründen bir üste çıkıp data'ya giriyoruz)
normal_path = '../data/Training_Data_Master'
attack_path = '../data/Attack_Data_Master'

# 2. Dataset Sınıfını Başlat
dataset = SyscallDataset(normal_path, attack_path)
print(f"Veri Setindeki Toplam Dosya (Süreç) Sayısı: {len(dataset)}")

# 3. DataLoader'ı Başlat (Veriyi 32'şerli paketler halinde GPU'ya hazırlayacak sistem)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn_pad)

# 4. Sistemi Test Et (Sadece ilk batch'i çekip boyutlarına bakıyoruz)
for sequences, labels in dataloader:
    print(f"Padded Batch Matris Boyutu: {sequences.shape}") 
    print(f"Etiket Matris Boyutu: {labels.shape}")
    
    # Matrisin içindeki paddingleri (sıfırları) görmek istersen:
    # print(sequences[0]) 
    break

In [4]:
import torch.nn as nn

# Model mimarisne padding_idx ekledik çünkü 0 da bir syscal olabileceği için yanlış eğitilir 0 koyarsak.
class SyscallLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SyscallLSTM, self).__init__()
        
        # 1. Katman: Embedding (Sayıları anlamlı vektörlere çevirir)
        # padding_idx=0 diyerek 0'ların bir syscall değil dolgu olduğunu belirtiyoruz
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # 2. Katman: LSTM (Sıralı ilişkiyi ve zamanı öğrenir)
        # batch_first=True diyerek verinin (32, Sekans_Uzunluğu) formatında olduğunu belirtiyoruz
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=2, dropout=0.2) # , num_layers=2, dropout=0.2 ekledik bize ne katar???
        
        # 3. Katman: Fully Connected / Linear (MLP katmanı gibi düşün)
        # LSTM'den gelen bilgiyi tek bir sayıya (0-1 arası olasılık için) indirger
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # x: [batch_size, seq_len]
    
        # 1. Hangi değerlerin padding OLMADIĞINI bul (0 olmayanlar)
        mask = (x != 0).float().unsqueeze(-1) # [batch_size, seq_len, 1]
        
        # 2. Embedding ve LSTM
        embedded = self.embedding(x)
        out, _ = self.lstm(embedded)
        
        # 3. MASKELENMİŞ ORTALAMA (Kritik Nokta!)
        # Sadece gerçek syscall'ların olduğu yerleri topla ve gerçek sayıya böl
        out_masked = out * mask
        out_sum = torch.sum(out_masked, dim=1)
        mask_sum = torch.sum(mask, dim=1).clamp(min=1e-9) # 0'a bölme hatasını önle
        
        mean_pooled = out_sum / mask_sum # Sinyal artık seyrelmiyor!
        
        return self.fc(mean_pooled)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Cihaz Belirleme (GPU varsa kullan, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sistem şu an {device} üzerinde çalışıyor.")

# 2. Hiperparametreler
VOCAB_SIZE = 2000 
EMBEDDING_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = 1

num_normal = 833  # Training_Data_Master içindeki dosya sayısı
num_attack = 746  # Attack_Data_Master içindeki dosya sayısı

# 3. Model, Loss ve Optimizer
model = SyscallLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
#criterion = nn.BCEWithLogitsLoss()
#pos_weight = torch.tensor([normal_count / attack_count]).to(device)
weight_value = torch.tensor([1.2]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=weight_value)

optimizer = optim.Adam(model.parameters(), lr=0.001)
"""
# Veri sayılarını manuel veya otomatik hesapla (ADFA-LD için yaklaşık oranlar)
num_normal = 833  # Training_Data_Master içindeki dosya sayısı
num_attack = 746  # Attack_Data_Master içindeki dosya sayısı
# Not: Eğer sayılar farklıysa sum(1 for l in dataset.labels if l == 0) ile hesapla

# Saldırı sınıfına verilecek ağırlık (Örn: 5.0 kat daha önemli gör)
weight_value = torch.tensor([5.0]).to(device)
#criterion = nn.BCEWithLogitsLoss(pos_weight=weight_value)
pos_weight = torch.tensor([normal_count / attack_count]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


model = SyscallLSTM(2000, 64, 128, 1).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

"""
print("--- Kurulum Tamamlandı ---")
print(model)

Sistem şu an cpu üzerinde çalışıyor.
--- Kurulum Tamamlandı ---
SyscallLSTM(
  (embedding): Embedding(2000, 64, padding_idx=0)
  (lstm): LSTM(64, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)


# Model eğitim kısmı

In [6]:
num_epochs = 40 # Tüm verinin üzerinden 10 kez geçeceğiz
print(f"Eğitim başlıyor ({len(train_dataset)} örnek ile)...")


for epoch in range(num_epochs):
    model.train() # Modeli eğitim moduna al
    total_loss = 0
    
    for sequences, labels in train_loader: # 
        # Verileri GPU/CPU cihazına taşı
        sequences = sequences.to(device)
        labels = labels.to(device)
        
        # 1. Gradyanları sıfırla
        optimizer.zero_grad()
        
        # 2. İleri besleme (Forward)
        outputs = model(sequences)
        
        # 3. Hatayı hesapla
        loss = criterion(outputs, labels)
        
        # 4. Geriye yayılım (Backward)
        loss.backward()
        
        # 5. Ağırlıkları güncelle
        optimizer.step()
        
        total_loss += loss.item()
        
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Ortalama Hata (Loss): {avg_loss:.4f}")

print("Eğitim Tamamlandı!")

Eğitim başlıyor (1262 örnek ile)...
Epoch [1/40], Ortalama Hata (Loss): 0.5361
Epoch [2/40], Ortalama Hata (Loss): 0.3402
Epoch [3/40], Ortalama Hata (Loss): 0.2992
Epoch [4/40], Ortalama Hata (Loss): 0.2936
Epoch [5/40], Ortalama Hata (Loss): 0.2377
Epoch [6/40], Ortalama Hata (Loss): 0.2119
Epoch [7/40], Ortalama Hata (Loss): 0.2054
Epoch [8/40], Ortalama Hata (Loss): 0.1613
Epoch [9/40], Ortalama Hata (Loss): 0.1688
Epoch [10/40], Ortalama Hata (Loss): 0.1884
Epoch [11/40], Ortalama Hata (Loss): 0.3215
Epoch [12/40], Ortalama Hata (Loss): 0.1747
Epoch [13/40], Ortalama Hata (Loss): 0.1460
Epoch [14/40], Ortalama Hata (Loss): 0.1338
Epoch [15/40], Ortalama Hata (Loss): 0.1313
Epoch [16/40], Ortalama Hata (Loss): 0.1210
Epoch [17/40], Ortalama Hata (Loss): 0.1011
Epoch [18/40], Ortalama Hata (Loss): 0.1222
Epoch [19/40], Ortalama Hata (Loss): 0.1058
Epoch [20/40], Ortalama Hata (Loss): 0.0917
Epoch [21/40], Ortalama Hata (Loss): 0.1260
Epoch [22/40], Ortalama Hata (Loss): 0.1066
Epoch

# pth ile model kaydetme

In [7]:
# Modeli kaydetmek için bir klasör oluştur (isteğe bağlı ama düzenli olur)
if not os.path.exists('../models'):
    os.makedirs('../models')

# Modeli kaydet
model_path = '../models/syscall_lstm_model.pth'
torch.save(model.state_dict(), model_path)

print(f"Model başarıyla kaydedildi: {model_path}")

Model başarıyla kaydedildi: ../models/syscall_lstm_model.pth


# safetensors ile model kaydetme

In [8]:
from safetensors.torch import save_file

# 1. Klasörün varlığından emin ol
if not os.path.exists('../models'):
    os.makedirs('../models')

# 2. Modelin ağırlıklarını (state_dict) al
state_dict = model.state_dict()

# 3. Safetensors olarak kaydet
save_path = "../models/syscall_lstm_model.safetensors"
save_file(state_dict, save_path)

print(f"Model Safetensors formatında başarıyla kaydedildi: {save_path}")


Model Safetensors formatında başarıyla kaydedildi: ../models/syscall_lstm_model.safetensors


# pth ile kaydedilen modeli yükleme


from safetensors.torch import load_file

# 1. Boş mimariyi oluştur
loaded_model = SyscallLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)

# 2. Safetensors dosyasını oku
tensors = load_file("../models/syscall_lstm_model.safetensors")

# 3. Ağırlıkları modele enjekte et
loaded_model.load_state_dict(tensors)
loaded_model.eval()

print("Güvenli (Safetensors) model başarıyla yüklendi!")

# safetensors ile kaydedilen modeli yükleme


# 1. Boş bir model nesnesi oluştur (Aynı mimari ile)
loaded_model = SyscallLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)

# 2. Kaydedilen ağırlıkları bu boş modele enjekte et
loaded_model.load_state_dict(torch.load('../models/syscall_lstm_model.pth', map_location=device))

# 3. Modeli test (evaluation) moduna al
loaded_model.eval()

print("Eğitilmiş model başarıyla yüklendi ve kullanıma hazır!")

In [9]:
import torch
from safetensors.torch import load_file
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader

# 1. Güvenli Modeli Yükle
# Not: Vocab_size ve diğer hiperparametrelerin eğitimdekiyle AYNI olması şarttır.
loaded_model = SyscallLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
tensors = load_file("../models/syscall_lstm_model.safetensors")
loaded_model.load_state_dict(tensors)
loaded_model.eval() # Modeli 'Değerlendirme' moduna al (Gradyan takibini kapatır)

print("--- Safetensors Model Başarıyla Yüklendi ---\n")


print(f"Test edilecek toplam örnek sayısı: {len(test_dataset)}")

# 3. Model Performansı Ölçme (Evaluation Loop)
all_preds = []
all_labels = []

print("Tahminler yürütülüyor...")

with torch.no_grad(): # Türev hesaplamayarak RAM ve hız tasarrufu sağlıyoruz
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        
        # Modeli çalıştır
        outputs = loaded_model(sequences)
        
        # Ham skoru (logit) 0.5 eşiğine göre 0 veya 1'e çevir
        preds = (torch.sigmoid(outputs) > 0.5).float()
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

# 4. Metriklerin Hesaplanması
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print("\n" + "="*30)
print("FINAL MODEL PERFORMANS RAPORU")
print("="*30)
print(f"Doğruluk (Accuracy): %{accuracy*100:.2f}")
print(f"Kesinlik (Precision): %{precision*100:.2f}  -> Saldırı dediklerimizin gerçeklik oranı")
print(f"Duyarlılık (Recall): %{recall*100:.2f}     -> Gerçek saldırıları yakalama oranı")
print(f"F1-Score: {f1:.4f}")
print("="*30)



--- Safetensors Model Başarıyla Yüklendi ---

Test edilecek toplam örnek sayısı: 317
Tahminler yürütülüyor...

FINAL MODEL PERFORMANS RAPORU
Doğruluk (Accuracy): %95.90
Kesinlik (Precision): %94.19  -> Saldırı dediklerimizin gerçeklik oranı
Duyarlılık (Recall): %97.33     -> Gerçek saldırıları yakalama oranı
F1-Score: 0.9574
